### Simplex Tableau
---

Construção do algoritmo Simplex na forma Tableau:

* Não precisamos dos passos dados no Simplex "raiz", as iterações são compostas apenas de operações matriciais
* No Tableau, criamos uma matriz indexada pelas variábeis básicas, composta pela matriz de restrições com o vetor $b$ concatenado, e o vetor de custo com o resultado $z$ concatenado, como mostra a figura abaixo:

<img src="files/simplex_tableau.jpg">

In [48]:
import numpy as np
A = np.matrix([[1,1,1,0,0], [2,1,0,1,0], [-1,1,0,0,1]])
c = np.array([2,3,0,0,0]).reshape(5,1)
#x = np.array([0,0,6,10,4]).reshape(5,1)
b = np.array([6,10,4]).reshape(3,1)

Passar para forma canônica:

$Ax = b \rightarrow Ax \leq b, Ax \geq b \rightarrow Ax - [x_{n+1}, ... x_{n+m}] = b, Ax + [x_{n+m}, ... x_{n+2m}] = b$ ...

In [49]:
A

matrix([[ 1,  1,  1,  0,  0],
        [ 2,  1,  0,  1,  0],
        [-1,  1,  0,  0,  1]])

In [50]:
b

array([[ 6],
       [10],
       [ 4]])

In [51]:
c

array([[2],
       [3],
       [0],
       [0],
       [0]])

In [74]:
T = np.concatenate((np.zeros(A.shape[0]).reshape(3,1), A, b), axis=1)
T = np.concatenate((np.array([1] + list(-c.T[0]) + [0]).reshape(1, 7), T), axis=0)
T

matrix([[ 1., -2., -3.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  1.,  0.,  0.,  6.],
        [ 0.,  2.,  1.,  0.,  1.,  0., 10.],
        [ 0., -1.,  1.,  0.,  0.,  1.,  4.]])

In [33]:
import numpy as np

class Simplex():
    
     """
    Instancia o metodo Simplex.

    Parameters
    ----------
    A : np.matrix
        Matriz de restricoes do PL
    c : np.array
        Vetor de custos do PL.
    x : np.array
        Vetor de solução inicial do PL.
    b : np.array
        Vetor de resposta do PL.

    Return
    ------
    x : np.array
        Solucao otima do PL (nx1).
    """
    def __init__(self, A, c, x, b):

        self.A = A
        self.c = c
        self.x = x
        self.b = b
        
    def calculate_tableau(self):
        
        # cria o tableau
        T = np.concatenate((np.zeros(A.shape[0]).reshape(3,1), A, b), axis=1)
        T = np.concatenate((np.array([1] + list(-c.T[0]) + [0]).reshape(1, 7), T), axis=0)

IndentationError: unindent does not match any outer indentation level (<ipython-input-33-fcc0a81f6251>, line 24)

### Exemplos
---

#### Fase I + Fase II